In [54]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"
data = requests.get(standings_url)
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
team_urls = [f"https://fbref.com{l}" for l in links]
data = requests.get(team_urls[0])


In [55]:
!pip install html5lib

In [56]:
team_urls

['https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/7c21e445/West-Ham-United-Stats',
 'https://fbref.com/en/squads/4ba7cbea/Bournemouth-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/d3fd31cc/Everton-Stats',
 'https://fbref.com/en/squads/

In [59]:
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

C:\Users\oguzo\AppData\Local\Temp\ipykernel_11048\1811192786.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


FeatureNotFound: Couldn't find a tree builder with the features you requested: html5lib. Do you need to install a parser library?

In [41]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

In [42]:
data = requests.get(f"https://fbref.com{links[0]}")

In [43]:
shooting = pd.read_html(data.text, match="Shooting")[0]

C:\Users\oguzo\AppData\Local\Temp\ipykernel_11048\2517714723.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


In [44]:
shooting.head(40)

For Liverpool                                                              \
            Date   Time            Comp         Round  Day Venue Result   GF   
0     2024-08-17  12:30  Premier League   Matchweek 1  Sat  Away      W  2.0   
1     2024-08-25  16:30  Premier League   Matchweek 2  Sun  Home      W  2.0   
2     2024-09-01  16:00  Premier League   Matchweek 3  Sun  Away      W  3.0   
3     2024-09-14  15:00  Premier League   Matchweek 4  Sat  Home      L  0.0   
4     2024-09-17  21:00    Champions Lg  League phase  Tue  Away      W  3.0   
5     2024-09-21  15:00  Premier League   Matchweek 5  Sat  Home      W  3.0   
6     2024-09-25  20:00         EFL Cup   Third round  Wed  Home      W  5.0   
7     2024-09-28  17:30  Premier League   Matchweek 6  Sat  Away      W  2.0   
8     2024-10-02  20:00    Champions Lg  League phase  Wed  Home      W  2.0   
9     2024-10-05  12:30  Premier League   Matchweek 7  Sat  Away      W  1.0   
10           NaN    NaN             NaN           NaN  NaN   NaN     --  NaN   

                          ... Standard               Expected               \
     GA         Opponent  ...     Dist   FK PK PKatt       xG npxG npxG/Sh   
0   0.0     Ipswich Town  ...     14.8  0.0  0     0      2.6  2.6    0.15   
1   0.0        Brentford  ...     13.6  1.0  0     0      2.5  2.5    0.14   
2   0.0   Manchester Utd  ...     13.4  0.0  0     0      1.8  1.8    0.16   
3   1.0  Nott'ham Forest  ...     14.9  0.0  0     0      0.9  0.9    0.07   
4   1.0         it Milan  ...     15.7  1.0  0     0      3.1  3.1    0.14   
5   0.0      Bournemouth  ...     16.6  0.0  0     0      2.0  2.0    0.11   
6   1.0         West Ham  ...      NaN  NaN  0     0      NaN  NaN     NaN   
7   1.0           Wolves  ...     18.0  1.0  1     1      2.5  1.7    0.19   
8   0.0       it Bologna  ...     15.4  0.0  0     0      1.2  1.2    0.13   
9   0.0   Crystal Palace  ...     18.9  0.0  0     0      1.4  1.4    0.09   
10  NaN              NaN  ...     15.6  3.0  1     1      NaN  NaN    0.13   

                 Unnamed: 25_level_0  
    G-xG np:G-xG        Match Report  
0   -0.6    -0.6        Match Report  
1   -0.5    -0.5        Match Report  
2    1.2     1.2        Match Report  
3   -0.9    -0.9        Match Report  
4   -0.1    -0.1        Match Report  
5    1.0     1.0        Match Report  
6    NaN     NaN        Match Report  
7   -0.5    -0.7        Match Report  
8    0.8     0.8        Match Report  
9   -0.4    -0.4        Match Report  
10  23.0    22.0                 NaN  

[11 rows x 26 columns]

In [45]:
shooting.columns = shooting.columns.droplevel()

In [46]:
team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")

In [47]:
team_data.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Opp Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2024-08-17,12:30,Premier League,Matchweek 1,Sat,Away,W,2.0,0.0,Ipswich Town,...,4-2-3-1,Tim Robinson,Match Report,NaN,18,5,14.8,0.0,0,0
1,2024-08-25,16:30,Premier League,Matchweek 2,Sun,Home,W,2.0,0.0,Brentford,...,4-4-2,Stuart Attwell,Match Report,NaN,19,8,13.6,1.0,0,0
2,2024-09-01,16:00,Premier League,Matchweek 3,Sun,Away,W,3.0,0.0,Manchester Utd,...,4-2-3-1,Anthony Taylor,Match Report,NaN,11,3,13.4,0.0,0,0
3,2024-09-14,15:00,Premier League,Matchweek 4,Sat,Home,L,0.0,1.0,Nott'ham Forest,...,4-2-3-1,Michael Oliver,Match Report,NaN,14,5,14.9,0.0,0,0
4,2024-09-17,21:00,Champions Lg,League phase,Tue,Away,W,3.0,1.0,it Milan,...,4-2-3-1,Espen Eskås,Match Report,NaN,23,11,15.7,1.0,0,0


In [48]:
years = list(range(2022, 2020, -1))
all_matches = []

In [49]:
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [51]:
import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

IndexError: list index out of range

In [59]:
len(all_matches)

39

In [60]:
match_df = pd.concat(all_matches)

In [61]:
match_df.columns = [c.lower() for c in match_df.columns]

In [62]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2021-08-15,16:30,Premier League,Matchweek 1,Sun,Away,L,0,1,Tottenham,...,Match Report,NaN,18.0,4.0,16.9,1.0,0.0,0.0,2022,Manchester City
2,2021-08-21,15:00,Premier League,Matchweek 2,Sat,Home,W,5,0,Norwich City,...,Match Report,NaN,16.0,4.0,17.3,1.0,0.0,0.0,2022,Manchester City
3,2021-08-28,12:30,Premier League,Matchweek 3,Sat,Home,W,5,0,Arsenal,...,Match Report,NaN,25.0,10.0,14.3,0.0,0.0,0.0,2022,Manchester City
4,2021-09-11,15:00,Premier League,Matchweek 4,Sat,Away,W,1,0,Leicester City,...,Match Report,NaN,25.0,8.0,14.0,0.0,0.0,0.0,2022,Manchester City
6,2021-09-18,15:00,Premier League,Matchweek 5,Sat,Home,D,0,0,Southampton,...,Match Report,NaN,16.0,1.0,15.7,1.0,0.0,0.0,2022,Manchester City
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0,4,Tottenham,...,Match Report,NaN,8.0,1.0,17.4,0.0,0.0,0.0,2021,Sheffield United
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0,2,Crystal Palace,...,Match Report,NaN,7.0,0.0,11.4,1.0,0.0,0.0,2021,Sheffield United
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1,0,Everton,...,Match Report,NaN,10.0,3.0,17.0,0.0,0.0,0.0,2021,Sheffield United
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0,1,Newcastle Utd,...,Match Report,NaN,11.0,1.0,16.0,1.0,0.0,0.0,2021,Sheffield United


In [27]:
match_df.to_csv("matches.csv")

NameError: name 'match_df' is not defined